In [1]:
#NBA Sportsdataverse
import csv
import requests
import sportsdataverse as sdv
import pandas as pd
import sqlite3


#Connecting to the database
sqlite_db = r"nba.db"
conn = sqlite3.connect(sqlite_db)
cur = conn.cursor()

In [2]:
#Importing the NBA Schedule
nba_schedule = sdv.nba.load_nba_schedule(2002,2023)

100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


In [6]:
#importing the NBA Boxscores, as the data has a different structure, we will import the data in two parts
boxscoredata = sdv.nba.load_nba_player_boxscore(seasons=range(2002,2013))
boxscoredata2 = sdv.nba.load_nba_player_boxscore(seasons=range(2014,2023))

100%|██████████| 9/9 [00:10<00:00,  1.22s/it]


In [9]:
#to merge the two data parts, we will drop the reason column from the second part of the data
boxscoredata2 = boxscoredata2.drop(["reason"])

In [10]:
#Transforming the data into a dataframe
boxscoredatadf = pd.DataFrame(boxscoredata)
boxscoredata2df = pd.DataFrame(boxscoredata2)


AttributeError: 'DataFrame' object has no attribute 'append'

In [11]:
#merging the two dataframes
import polars as pl

# Convert the dataframes to polars DataFrames
boxscoredata_df = pl.from_pandas(boxscoredata.to_pandas())
boxscoredata2_df = pl.from_pandas(boxscoredata2.to_pandas())

# Concatenate the dataframes vertically
combined_df = pl.concat([boxscoredata_df, boxscoredata2_df])

# Print the combined dataframe
print(combined_df.head())

pandascombdf = combined_df.to_pandas()

print(pandascombdf.head())


shape: (5, 56)
┌───────────┬────────┬────────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ game_id   ┆ season ┆ season_typ ┆ game_date  ┆ … ┆ opponent_ ┆ opponent_ ┆ opponent_ ┆ opponent_ │
│ ---       ┆ ---    ┆ e          ┆ ---        ┆   ┆ team_logo ┆ team_colo ┆ team_alte ┆ team_scor │
│ i32       ┆ i32    ┆ ---        ┆ datetime[n ┆   ┆ ---       ┆ r         ┆ rnate_col ┆ e         │
│           ┆        ┆ i32        ┆ s]         ┆   ┆ str       ┆ ---       ┆ or        ┆ ---       │
│           ┆        ┆            ┆            ┆   ┆           ┆ str       ┆ ---       ┆ i32       │
│           ┆        ┆            ┆            ┆   ┆           ┆           ┆ str       ┆           │
╞═══════════╪════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 220612017 ┆ 2002   ┆ 3          ┆ 2002-06-12 ┆ … ┆ null      ┆ 06143F    ┆ ffffff    ┆ 107       │
│           ┆        ┆            ┆ 00:00:00   ┆   ┆           ┆           ┆

In [12]:
#Exporting the final dataframe to the database
pandascombdf.to_sql('nba_player_boxscore', conn, if_exists='replace', index = False)

675114